
# Vienna Coffee House in Madrid

## 1. Introduction / Business problem

In Madrid, if someone were looking to open a typical viennese coffee house, the question is, where would you recommend to open it? 

The background of the problem is that in order for the coffee house to be profitable, there must be enough customers, and in order to have enough customers, it is not worth setting up one in the immediate promixity of existing other ones.

Lets also make sure that the location where we open the coffee house will have the highest amount of customers possible. The New owner should care about this problem because the location of the new coffee house has a significant impact on the expected returns.

## 2. Data Preparation

We will use a CSV file with data extracted from Wikipedia with the information of each of the 21 districts in Madrid: Population, Latitute and Longitude.
Source: https://en.wikipedia.org/wiki/Districts_of_Madrid

In [86]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe
!pip install folium
import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

We will use a CSV file with data from Wikipedia with the information of each of the 21 districts in Madrid: Population, Latitude and Longitude

In [87]:
# The code was removed by Watson Studio for sharing.

,District Number,District,latitude,longitude,population
0,1,Centro,40.41831,-3.70275,131928
1,2,Arganzuela,40.40021,-3.69618,151965
2,3,Retiro,40.41317,-3.68307,118516
3,4,Salamanca,40.42972,-3.67975,143800
4,5,Chamartin,40.46206,-3.67660,143424


Add the population weightage for each district

In [112]:
total_population = madrid['population'].sum()
#print (total_population)

# the higher populated district will have the higher score
madrid['Population Score'] = madrid['population']/total_population*100

# sort the dataframe based on the descending population score
madrid = madrid.sort_values('Population Score', ascending=False)
madrid.head(10)

,District Number,District,latitude,longitude,population,Population Score
10,11,Carabanchel,40.39094,-3.72420,243998,7.667649
7,8,Fuencarral-El Pardo,40.49840,-3.73140,238756,7.502919
9,10,Latina,40.38897,-3.74569,233808,7.347427
12,13,Puente de Vallecas,40.39354,-3.66200,227595,7.152184
14,15,Ciudad Lineal,40.44505,-3.65132,212529,6.678734
15,16,Hortaleza,40.47444,-3.64110,180462,5.671027
19,20,San Blas-Canillejas,40.43893,-3.61537,154357,4.850676
5,6,Tetuan,40.45975,-3.69750,153789,4.832827
1,2,Arganzuela,40.40021,-3.69618,151965,4.775507
3,4,Salamanca,40.42972,-3.67975,143800,4.518922


Location of the districts in the Madrid Map

In [78]:
address = "Madrid, ON"
madrid_data = madrid
geolocator = Nominatim(user_agent="madrid_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid city is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid city is 40.457473750000005, -3.4696396792893456.


In [79]:
# create map of Madrid using latitude and longitude values
madrid = folium.Map(location=[latitude, longitude], zoom_start=10)
madrid

In [88]:
for lat, lng, District in zip(
        madrid['latitude'], 
        madrid['longitude'], 
        madrid['District']): 
    label = '{}'.format(District)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_madrid)  

map_madrid

Use of Foursquare API to find out the venue category.

In [30]:
CLIENT_ID = 'XY0ETEUVLYZRKBKRBPMWZS4NZW52RZZRZCMQFHS0PGA0G5BB' # your Foursquare ID
CLIENT_SECRET = 'IY0Z2Q0BH1I3VXLMMOGCA2QBWQSGSAG31A43YHV201EVNBGW' # your Foursquare Secret
ACCESS_TOKEN = 'N3C5IY5V0J3U4TC53G5ML3T3EAFWFYFZJYSRX2PQANSKSCAY' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XY0ETEUVLYZRKBKRBPMWZS4NZW52RZZRZCMQFHS0PGA0G5BB
CLIENT_SECRET:IY0Z2Q0BH1I3VXLMMOGCA2QBWQSGSAG31A43YHV201EVNBGW


In [46]:
search_query = 'café'
radius = 500
print(search_query + ' .... OK!')

café .... OK!


In [34]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=XY0ETEUVLYZRKBKRBPMWZS4NZW52RZZRZCMQFHS0PGA0G5BB&client_secret=IY0Z2Q0BH1I3VXLMMOGCA2QBWQSGSAG31A43YHV201EVNBGW&ll=40.457473750000005,-3.4696396792893456&oauth_token=N3C5IY5V0J3U4TC53G5ML3T3EAFWFYFZJYSRX2PQANSKSCAY&v=20180604&query=coffee&radius=500&limit=30'

In [90]:
madrid.loc[1, 'District']
madrid.head(1)

,District Number,District,latitude,longitude,population
0,1,Centro,40.41831,-3.70275,131928


In [92]:
district_latitude = madrid.loc[1, 'latitude'] # district latitude value
district_longitude = madrid.loc[1, 'longitude'] # district longitude value

district_name = madrid.loc[1, 'District'] # district name

print('Latitude and longitude values of {} are {}, {}.'.format(district_name, 
                                                               district_latitude, 
                                                               district_longitude))

Latitude and longitude values of Arganzuela are 40.400209999999994, -3.6961800000000005.


In [93]:
# explore the top 100 venues
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    district_latitude, 
    district_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=XY0ETEUVLYZRKBKRBPMWZS4NZW52RZZRZCMQFHS0PGA0G5BB&client_secret=IY0Z2Q0BH1I3VXLMMOGCA2QBWQSGSAG31A43YHV201EVNBGW&v=20180604&ll=40.400209999999994,-3.6961800000000005&radius=500&limit=100'

In [94]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fce6612de8ff27daff13ed1'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Delicias',
  'headerFullLocation': 'Delicias, Madrid',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 51,
  'suggestedBounds': {'ne': {'lat': 40.4047100045, 'lng': -3.6902819165509313},
   'sw': {'lat': 40.39570999549999, 'lng': -3.7020780834490696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b83c5dff964a520e10f31e3',
       'name': 'La Pequeña Graná',
       'location': {'address': 'C. General Palanca, 16',
        'lat': 40.39957392916725,
        'lng': -3.6985499739321477,
        'labeledLatLngs': [{'label': 'display',
          'lat': 40.39

In [41]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [45]:
# clean the json and structure it into a pandas dataframe.
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(20)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,lat,lng
0,La Pequeña Graná,Tapas Restaurant,40.399574,-3.698550
1,Havana Blues,Cuban Restaurant,40.402050,-3.698488
2,Tres Cerditos,Chinese Restaurant,40.397316,-3.694184
3,Mercado de Santa María de la Cabeza,Farmers Market,40.402860,-3.696819
4,Salón de Té Al Yabal,Cocktail Bar,40.399015,-3.700249
5,Garaje Lumiere,Movie Theater,40.400047,-3.695005
6,Mercado de Motores,Flea Market,40.399149,-3.691978
7,PanArte,Bakery,40.399279,-3.694182
8,Bululu2120,Trade School,40.401660,-3.696455
9,El Quinto Pecado,Gastropub,40.400028,-3.694446


In [95]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

51 venues were returned by Foursquare.


In [96]:
# Explore districts
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [99]:
madrid_venues = getNearbyVenues(names= madrid['District'],
                                   latitudes= madrid['latitude'],
                                   longitudes= madrid['longitude']
                                  )

print(madrid_venues.shape)
madrid_venues.head(10)

Centro
Arganzuela
Retiro
Salamanca
Chamartin
Tetuan
Chamberi
Fuencarral-El Pardo
Moncloa-Aravaca
Latina
Carabanchel
Usera
Puente de Vallecas
Moratalaz
Ciudad Lineal
Hortaleza
Villaverde
Villa de Vallecas
Vicalvaro
San Blas-Canillejas
Barajas
(575, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Centro,40.41831,-3.70275,LUSH,40.419012,-3.704898,Cosmetics Shop
1,Centro,40.41831,-3.70275,SunGate Hostel,40.418696,-3.704623,Hostel
2,Centro,40.41831,-3.70275,Club del Gourmet Corte Ingles,40.417497,-3.704686,Gourmet Shop
3,Centro,40.41831,-3.70275,La Pulpería de Victoria,40.416506,-3.701709,Seafood Restaurant
4,Centro,40.41831,-3.70275,Apple Puerta del Sol,40.416898,-3.702065,Electronics Store
5,Centro,40.41831,-3.70275,TAKOS,40.418938,-3.703748,Mexican Restaurant
6,Centro,40.41831,-3.70275,Kiko store,40.420128,-3.702317,Cosmetics Shop
7,Centro,40.41831,-3.70275,Casa del Libro,40.420101,-3.703193,Bookstore
8,Centro,40.41831,-3.70275,Puerta del Sol,40.417034,-3.705251,Plaza
9,Centro,40.41831,-3.70275,Fnac,40.419176,-3.705269,Electronics Store


In [100]:
# find out how many unique categories
madrid_venues.groupby('District').count()

,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
District,,,,,,
Arganzuela,51,51,51,51,51,51
Barajas,8,8,8,8,8,8
Carabanchel,11,11,11,11,11,11
Centro,79,79,79,79,79,79
Chamartin,24,24,24,24,24,24
Chamberi,100,100,100,100,100,100
Ciudad Lineal,25,25,25,25,25,25
Hortaleza,15,15,15,15,15,15
Latina,15,15,15,15,15,15


In [101]:
# analyze each district

# one hot encoding
madrid_onehot = pd.get_dummies(madrid_venues[['Venue Category']], prefix="", prefix_sep="")

# add district column back to dataframe
madrid_onehot['District'] = madrid_venues['District'] 

# move district column to the first column
fixed_columns = [madrid_onehot.columns[-1]] + list(madrid_onehot.columns[:-1])
madrid_onehot = madrid_onehot[fixed_columns]

madrid_onehot.head()

,District,Accessories Store,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,...,Tattoo Parlor,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trade School,Train Station,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop
0,Centro,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Centro,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Centro,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Centro,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Centro,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [102]:
# group rows by district and by taking the mean of the frequency of occurrence of each category
madrid_grouped = madrid_onehot.groupby('District').mean().reset_index()
madrid_grouped.head(10)

,District,Accessories Store,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,...,Tattoo Parlor,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trade School,Train Station,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop
0,Arganzuela,0.000000,0.00,0.000000,0.039216,0.000000,0.0,0.000000,0.00,0.00,...,0.019608,0.019608,0.000000,0.000000,0.00,0.019608,0.019608,0.00,0.000000,0.0
1,Barajas,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0
2,Carabanchel,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0
3,Centro,0.012658,0.00,0.000000,0.000000,0.012658,0.0,0.025316,0.00,0.00,...,0.000000,0.000000,0.025316,0.000000,0.00,0.000000,0.000000,0.00,0.012658,0.0
4,Chamartin,0.000000,0.00,0.041667,0.000000,0.000000,0.0,0.000000,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0
5,Chamberi,0.000000,0.01,0.000000,0.000000,0.000000,0.0,0.000000,0.01,0.00,...,0.000000,0.000000,0.010000,0.000000,0.01,0.000000,0.000000,0.01,0.010000,0.0
6,Ciudad Lineal,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.04,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0
7,Hortaleza,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0
8,Latina,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.00,...,0.000000,0.000000,0.000000,0.066667,0.00,0.000000,0.000000,0.00,0.000000,0.0
9,Moncloa-Aravaca,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0


Find out the top 10 most common venues in the neighbourhood.

In [103]:
# print each district along with the top 5 most common venues
num_top_venues = 5

for hood in madrid_grouped['District']:
    #print("----"+hood+"----")
    temp = madrid_grouped[madrid_grouped['District'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    #print('\n')

In [104]:
# sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [105]:
madrid_grouped.head()

,District,Accessories Store,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,...,Tattoo Parlor,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trade School,Train Station,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop
0,Arganzuela,0.000000,0.0,0.000000,0.039216,0.000000,0.0,0.000000,0.0,0.0,...,0.019608,0.019608,0.000000,0.0,0.0,0.019608,0.019608,0.0,0.000000,0.0
1,Barajas,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
2,Carabanchel,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
3,Centro,0.012658,0.0,0.000000,0.000000,0.012658,0.0,0.025316,0.0,0.0,...,0.000000,0.000000,0.025316,0.0,0.0,0.000000,0.000000,0.0,0.012658,0.0
4,Chamartin,0.000000,0.0,0.041667,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0


 Explore the top 5 venues for each district.

In [106]:
# create the new dataframe and display the top 5 venues for each district.
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
District_venues_sorted = pd.DataFrame(columns=columns)
District_venues_sorted['District'] = madrid_grouped['District']

for ind in np.arange(madrid_grouped.shape[0]):
    District_venues_sorted.iloc[ind, 1:] = return_most_common_venues(madrid_grouped.iloc[ind, :], num_top_venues)

District_venues_sorted.head(21)

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Arganzuela,Tapas Restaurant,Spanish Restaurant,Grocery Store,Restaurant,Bakery
1,Barajas,Coffee Shop,Tapas Restaurant,Food & Drink Shop,Mexican Restaurant,Café
2,Carabanchel,Coffee Shop,Grocery Store,Fast Food Restaurant,Farmers Market,Supermarket
3,Centro,Plaza,Tapas Restaurant,Hotel,Gourmet Shop,Café
4,Chamartin,Restaurant,Cocktail Bar,Tapas Restaurant,Spanish Restaurant,Gym
5,Chamberi,Restaurant,Spanish Restaurant,Bar,Tapas Restaurant,Brewery
6,Ciudad Lineal,Spanish Restaurant,Grocery Store,Gym / Fitness Center,Hotel,Pizza Place
7,Hortaleza,Spanish Restaurant,Food,Tapas Restaurant,Metro Station,Bar
8,Latina,Grocery Store,Fast Food Restaurant,Breakfast Spot,Comedy Club,Coffee Shop
9,Moncloa-Aravaca,Spanish Restaurant,Gym,Pool,Pub,Wine Shop


Add the Coffee Shop weighted score for each district. Also add score for Breakfast Spot as they are similar type of place.

In [107]:
District_venues_sorted['Coffee Score'] = 0
venue1 = 'Coffee Shop'
venue_count = 0

# award the score based on the common findings
District_venues_sorted.loc[District_venues_sorted['1st Most Common Venue'].str.contains(venue1), ['Coffee Score']] = 1.0
District_venues_sorted.loc[District_venues_sorted['2nd Most Common Venue'].str.contains(venue1), ['Coffee Score']] = 0.8
District_venues_sorted.loc[District_venues_sorted['3rd Most Common Venue'].str.contains(venue1), ['Coffee Score']] = 0.6
District_venues_sorted.loc[District_venues_sorted['4th Most Common Venue'].str.contains(venue1), ['Coffee Score']] = 0.4
District_venues_sorted.loc[District_venues_sorted['5th Most Common Venue'].str.contains(venue1), ['Coffee Score']] = 0.2



In [108]:
District_venues_sorted['Breakfast Spot Score'] = 0
venue2 = 'Breakfast Spot'
venue_count = 0

# award the score based on the common findings
District_venues_sorted.loc[District_venues_sorted['1st Most Common Venue'].str.contains(venue2), ['Breakfast Spot Score']] = 1.0
District_venues_sorted.loc[District_venues_sorted['2nd Most Common Venue'].str.contains(venue2), ['Breakfast Spot Score']] = 0.8
District_venues_sorted.loc[District_venues_sorted['3rd Most Common Venue'].str.contains(venue2), ['Breakfast Spot Score']] = 0.6
District_venues_sorted.loc[District_venues_sorted['4th Most Common Venue'].str.contains(venue2), ['Breakfast Spot Score']] = 0.4
District_venues_sorted.loc[District_venues_sorted['5th Most Common Venue'].str.contains(venue2), ['Breakfast Spot Score']] = 0.2


In [117]:
# merge the two dataframes
merged_data = pd.merge(madrid , District_venues_sorted, on='District')

# clean up the merged dataset
merged_data = merged_data.drop('1st Most Common Venue', 1)
merged_data = merged_data.drop('2nd Most Common Venue', 1)
merged_data = merged_data.drop('3rd Most Common Venue', 1)
merged_data = merged_data.drop('4th Most Common Venue', 1)
merged_data = merged_data.drop('5th Most Common Venue', 1)

merged_data.head(10)


,District Number,District,latitude,longitude,population,Population Score,Coffee Score,Breakfast Spot Score
0,11,Carabanchel,40.39094,-3.72420,243998,7.667649,1.0,0.0
1,10,Latina,40.38897,-3.74569,233808,7.347427,0.2,0.6
2,13,Puente de Vallecas,40.39354,-3.66200,227595,7.152184,0.0,0.0
3,15,Ciudad Lineal,40.44505,-3.65132,212529,6.678734,0.0,0.0
4,16,Hortaleza,40.47444,-3.64110,180462,5.671027,0.0,0.0
5,20,San Blas-Canillejas,40.43893,-3.61537,154357,4.850676,0.0,0.0
6,6,Tetuan,40.45975,-3.69750,153789,4.832827,0.0,0.0
7,2,Arganzuela,40.40021,-3.69618,151965,4.775507,0.0,0.0
8,4,Salamanca,40.42972,-3.67975,143800,4.518922,0.2,0.0
9,5,Chamartin,40.46206,-3.67660,143424,4.507106,0.0,0.0


Calculate the total weighted score for each district.

In [119]:

# considering this to be a fair weightage on the three parameters
population_weight = 0.5
coffee_score_weight = 0.3
breakfast_spot_weight = 0.2

# calculate the total score
merged_data['Total Score'] = ((merged_data['Population Score']*population_weight) + (merged_data['Coffee Score']*coffee_score_weight) + (merged_data['Breakfast Spot Score']*breakfast_spot_weight))

# sort the dataframe based on the descending total score
merged_data = merged_data.sort_values('Total Score', ascending=False)

## 3. Metodology: Cluster the districs using K-Clustering.

In [121]:
# set number of clusters
kclusters = 4

#madrid_grouped_clustering = madrid_grouped.drop('District', 1)

# clean up data further
madrid_grouped_clustering = merged_data.drop('District', 1)

madrid_grouped_clustering.head(10)

,District Number,latitude,longitude,population,Population Score,Coffee Score,Breakfast Spot Score,Total Score
0,11,40.39094,-3.72420,243998,7.667649,1.0,0.0,4.133824
1,10,40.38897,-3.74569,233808,7.347427,0.2,0.6,3.853714
2,13,40.39354,-3.66200,227595,7.152184,0.0,0.0,3.576092
3,15,40.44505,-3.65132,212529,6.678734,0.0,0.0,3.339367
4,16,40.47444,-3.64110,180462,5.671027,0.0,0.0,2.835513
5,20,40.43893,-3.61537,154357,4.850676,0.0,0.0,2.425338
6,6,40.45975,-3.69750,153789,4.832827,0.0,0.0,2.416413
7,2,40.40021,-3.69618,151965,4.775507,0.0,0.0,2.387754
8,4,40.42972,-3.67975,143800,4.518922,0.2,0.0,2.319461
12,12,40.38866,-3.70035,134791,4.235814,0.6,0.0,2.297907


In [122]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(madrid_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 0, 0, 0, 0, 0, 0], dtype=int32)

In [124]:
# empty the invalid data
#merged_data['District'] = District_venues_sorted['District']
merged_data.isnull().sum()
merged_data.dropna(how = 'any', axis = 0, inplace = True)
print(merged_data.shape)
print(District_venues_sorted.shape)

(20, 9)
(20, 8)


In [125]:
merged_data.head()

,District Number,District,latitude,longitude,population,Population Score,Coffee Score,Breakfast Spot Score,Total Score
0,11,Carabanchel,40.39094,-3.72420,243998,7.667649,1.0,0.0,4.133824
1,10,Latina,40.38897,-3.74569,233808,7.347427,0.2,0.6,3.853714
2,13,Puente de Vallecas,40.39354,-3.66200,227595,7.152184,0.0,0.0,3.576092
3,15,Ciudad Lineal,40.44505,-3.65132,212529,6.678734,0.0,0.0,3.339367
4,16,Hortaleza,40.47444,-3.64110,180462,5.671027,0.0,0.0,2.835513


In [126]:
District_venues_sorted = District_venues_sorted.drop('Coffee Score', 1)
District_venues_sorted = District_venues_sorted.drop('Breakfast Spot Score', 1)
District_venues_sorted.head()

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Arganzuela,Tapas Restaurant,Spanish Restaurant,Grocery Store,Restaurant,Bakery
1,Barajas,Coffee Shop,Tapas Restaurant,Food & Drink Shop,Mexican Restaurant,Café
2,Carabanchel,Coffee Shop,Grocery Store,Fast Food Restaurant,Farmers Market,Supermarket
3,Centro,Plaza,Tapas Restaurant,Hotel,Gourmet Shop,Café
4,Chamartin,Restaurant,Cocktail Bar,Tapas Restaurant,Spanish Restaurant,Gym


In [127]:
madrid_merged = madrid
# add clustering labels
merged_data['Cluster Labels'] = kmeans.labels_

# merge madrid_grouped with toronto_data to add latitude/longitude for each district
madrid_merged = merged_data.join(District_venues_sorted.set_index('District'), on='District')

madrid_merged.head(10) 

,District Number,District,latitude,longitude,population,Population Score,Coffee Score,Breakfast Spot Score,Total Score,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,11,Carabanchel,40.39094,-3.72420,243998,7.667649,1.0,0.0,4.133824,1,Coffee Shop,Grocery Store,Fast Food Restaurant,Farmers Market,Supermarket
1,10,Latina,40.38897,-3.74569,233808,7.347427,0.2,0.6,3.853714,1,Grocery Store,Fast Food Restaurant,Breakfast Spot,Comedy Club,Coffee Shop
2,13,Puente de Vallecas,40.39354,-3.66200,227595,7.152184,0.0,0.0,3.576092,1,Gym,Pizza Place,Pub,Chinese Restaurant,Italian Restaurant
3,15,Ciudad Lineal,40.44505,-3.65132,212529,6.678734,0.0,0.0,3.339367,1,Spanish Restaurant,Grocery Store,Gym / Fitness Center,Hotel,Pizza Place
4,16,Hortaleza,40.47444,-3.64110,180462,5.671027,0.0,0.0,2.835513,0,Spanish Restaurant,Food,Tapas Restaurant,Metro Station,Bar
5,20,San Blas-Canillejas,40.43893,-3.61537,154357,4.850676,0.0,0.0,2.425338,0,Spanish Restaurant,Rock Club,Plaza,Park,Pizza Place
6,6,Tetuan,40.45975,-3.69750,153789,4.832827,0.0,0.0,2.416413,0,Spanish Restaurant,Restaurant,Chinese Restaurant,Supermarket,Hotel
7,2,Arganzuela,40.40021,-3.69618,151965,4.775507,0.0,0.0,2.387754,0,Tapas Restaurant,Spanish Restaurant,Grocery Store,Restaurant,Bakery
8,4,Salamanca,40.42972,-3.67975,143800,4.518922,0.2,0.0,2.319461,0,Restaurant,Spanish Restaurant,Boutique,Tapas Restaurant,Coffee Shop
12,12,Usera,40.38866,-3.70035,134791,4.235814,0.6,0.0,2.297907,0,Spanish Restaurant,Clothing Store,Coffee Shop,Gastropub,Beer Garden


Draw the cluster map.

In [128]:
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(madrid_merged['latitude'], madrid_merged['longitude'], madrid_merged['District'], madrid_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Results

In [129]:
# display the top 5 districts
target = merged_data.head(5)
target.head()

,District Number,District,latitude,longitude,population,Population Score,Coffee Score,Breakfast Spot Score,Total Score,Cluster Labels
0,11,Carabanchel,40.39094,-3.72420,243998,7.667649,1.0,0.0,4.133824,1
1,10,Latina,40.38897,-3.74569,233808,7.347427,0.2,0.6,3.853714,1
2,13,Puente de Vallecas,40.39354,-3.66200,227595,7.152184,0.0,0.0,3.576092,1
3,15,Ciudad Lineal,40.44505,-3.65132,212529,6.678734,0.0,0.0,3.339367,1
4,16,Hortaleza,40.47444,-3.64110,180462,5.671027,0.0,0.0,2.835513,0


In [130]:
# Cluster 1 
madrid_merged.loc[madrid_merged['Cluster Labels'] == 0, madrid_merged.columns[[0] + [1] + list(range(4, madrid_merged.shape[1]))]].sort_values('Population Score', ascending=False).head(5)

,District Number,District,population,Population Score,Coffee Score,Breakfast Spot Score,Total Score,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,16,Hortaleza,180462,5.671027,0.0,0.0,2.835513,0,Spanish Restaurant,Food,Tapas Restaurant,Metro Station,Bar
5,20,San Blas-Canillejas,154357,4.850676,0.0,0.0,2.425338,0,Spanish Restaurant,Rock Club,Plaza,Park,Pizza Place
6,6,Tetuan,153789,4.832827,0.0,0.0,2.416413,0,Spanish Restaurant,Restaurant,Chinese Restaurant,Supermarket,Hotel
7,2,Arganzuela,151965,4.775507,0.0,0.0,2.387754,0,Tapas Restaurant,Spanish Restaurant,Grocery Store,Restaurant,Bakery
8,4,Salamanca,143800,4.518922,0.2,0.0,2.319461,0,Restaurant,Spanish Restaurant,Boutique,Tapas Restaurant,Coffee Shop


In [131]:
# Cluster 2 
madrid_merged.loc[madrid_merged['Cluster Labels'] == 1, madrid_merged.columns[[0] + [1] + list(range(4, madrid_merged.shape[1]))]].sort_values('Population Score', ascending=False).head(5)

,District Number,District,population,Population Score,Coffee Score,Breakfast Spot Score,Total Score,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,11,Carabanchel,243998,7.667649,1.0,0.0,4.133824,1,Coffee Shop,Grocery Store,Fast Food Restaurant,Farmers Market,Supermarket
1,10,Latina,233808,7.347427,0.2,0.6,3.853714,1,Grocery Store,Fast Food Restaurant,Breakfast Spot,Comedy Club,Coffee Shop
2,13,Puente de Vallecas,227595,7.152184,0.0,0.0,3.576092,1,Gym,Pizza Place,Pub,Chinese Restaurant,Italian Restaurant
3,15,Ciudad Lineal,212529,6.678734,0.0,0.0,3.339367,1,Spanish Restaurant,Grocery Store,Gym / Fitness Center,Hotel,Pizza Place


In [132]:
# Cluster 3 
madrid_merged.loc[madrid_merged['Cluster Labels'] == 2, madrid_merged.columns[[0] + [1] + list(range(4, madrid_merged.shape[1]))]].sort_values('Population Score', ascending=False).head(5)

,District Number,District,population,Population Score,Coffee Score,Breakfast Spot Score,Total Score,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
18,19,Vicalvaro,70051,2.201356,1.0,0.8,1.560678,2,Coffee Shop,Breakfast Spot,Bar,Dog Run,Wine Shop
19,21,Barajas,46876,1.473081,1.0,0.0,1.036540,2,Coffee Shop,Tapas Restaurant,Food & Drink Shop,Mexican Restaurant,Café


In [133]:
# Cluster 4 
madrid_merged.loc[madrid_merged['Cluster Labels'] == 3, madrid_merged.columns[[0] + [1] + list(range(4, madrid_merged.shape[1]))]].sort_values('Population Score', ascending=False).head(5)

,District Number,District,population,Population Score,Coffee Score,Breakfast Spot Score,Total Score,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
14,3,Retiro,118516,3.724371,0.0,0.0,1.862185,3,Garden,Café,Plaza,Monument / Landmark,Park
15,9,Moncloa-Aravaca,116903,3.673682,0.0,0.0,1.836841,3,Spanish Restaurant,Gym,Pool,Pub,Wine Shop
16,18,Villa de Vallecas,104421,3.281435,0.0,0.0,1.640717,3,Gym,Pizza Place,Pub,Chinese Restaurant,Italian Restaurant
17,14,Moratalaz,94197,2.960145,0.0,0.0,1.480073,3,Bar,Plaza,Bakery,Falafel Restaurant,Metro Station


Selecting the best result from cluster 2: Carabanchel district, now we use a heatmap based kernel density estimation as a Folium plugin. With that we can have a possible good location for our new coffee shop.

In [216]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180604',
    "section": "Coffee Shop",
    "near": "Carabanchel",
    "radius": 1000,
    "limit": 50}
data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [217]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'totalResults', 'suggestedBounds', 'groups'])

In [218]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('unknown', 'Current map view', 'Current map view')

In [219]:

d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 40.37343216800195, 'lng': -3.7393510268310624},
  'sw': {'lat': 40.36091382192523, 'lng': -3.7597690507197137}},
 23)

In [203]:
d["geocode"]

{'what': '',
 'where': 'carabanchel',
 'center': {'lat': 40.36797, 'lng': -3.75003},
 'displayString': 'Carabanchel Alto, Madrid, Spain',
 'cc': 'ES',
 'geometry': {'bounds': {'ne': {'lat': 40.380486, 'lng': -3.746337},
   'sw': {'lat': 40.363338, 'lng': -3.762538}}},
 'longId': '72057594041054389'}

In [204]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [205]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [220]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 23


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4b0470d2f964a5202e5422e3',
  'name': 'Don Antonio',
  'location': {'address': 'Avenida de Carabanchel Alto',
   'lat': 40.370374502946405,
   'lng': -3.75331615892289,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.370374502946405,
     'lng': -3.75331615892289}],
   'cc': 'ES',
   'city': 'Madrid',
   'state': 'Madrid',
   'country': 'España',
   'formattedAddress': ['Avenida de Carabanchel Alto',
    'Madrid Madrid',
    'España']},
  'categories': [{'id': '4bf58dd8d48988d1df931735',
    'name': 'BBQ Joint',
    'pluralName': 'BBQ Joints',
    'shortName': 'BBQ',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bbqalt_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-4b0470d2f964a5202e5422e3-0'}

In [221]:
items[1]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4b794b50f964a520a9f22ee3',
  'name': 'Mesón Avelino',
  'location': {'address': 'C. Joaquín Turina, 42',
   'lat': 40.372219553419384,
   'lng': -3.755117241579208,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.372219553419384,
     'lng': -3.755117241579208}],
   'postalCode': '28044',
   'cc': 'ES',
   'city': 'Madrid',
   'state': 'Madrid',
   'country': 'España',
   'formattedAddress': ['C. Joaquín Turina, 42',
    '28044 Madrid Madrid',
    'España']},
  'categories': [{'id': '4bf58dd8d48988d150941735',
    'name': 'Spanish Restaurant',
    'pluralName': 'Spanish Restaurants',
    'shortName': 'Spanish',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/spanish_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-4b794b50f964a520a9f22e

In [222]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address = location["formattedAddress"]
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i coffee shop" % len(df))
df.head()

found 15 coffee shop


,uid,name,shortname,address,postalcode,lat,lng
0,4b794b50f964a520a9f22ee3,Mesón Avelino,Spanish,"[C. Joaquín Turina, 42, 28044 Madrid Madrid, E...",28044,40.372220,-3.755117
1,4d0ce543e0b98cfa403fd893,Lidl,Supermarket,"[Antonia Rodriguez Sacristan, 16, 28044 Madrid...",28044,40.372138,-3.743620
2,4c3c5281933b0f4763d7e321,Medina 3,Candy Store,"[Calle Aguacate, 42, LOC;DUPDO, 28044 Madrid M...",28044,40.371001,-3.746794
3,4cb35bae1463a143d6f7b3a9,Polideportivo Amorós,Gym,"[C. Joaquín Turina, 46 (Camino de las Cruces),...",28054,40.371419,-3.757047
4,4bca36a868f976b084f15e83,La Galería Café,Bar,"[C. Morales, 10, 28054 Madrid Madrid, España]",28054,40.361483,-3.756160


In [223]:
carabanchel = d["geocode"]["center"]
carabanchel

{'lat': 40.36797, 'lng': -3.75003}

In [225]:
from folium import plugins
# create map of Carabanchel using latitude and longitude values
map_carabanchel = folium.Map(location=[carabanchel["lat"], carabanchel["lng"]], zoom_start=10)
def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_carabanchel)

map_carabanchel

In [215]:
lat = 40.3716619 
lng = -3.7457912
map_carabanchel = folium.Map(location=[lat, lng], zoom_start=14)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="Our Coffee Shop!",
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_carabanchel)
map_carabanchel

## 4. Results

Based on the result above, the third cluster looks to offer a higher number of districts with good population score and low competition from others coffee shops and breakfast places.

In this 2nd cluster we recommend the Carabanchel district to open the first viennese coffee shop because it has the highest population of the entire cluster. Then to expand the bussiness the owner can consider other districts from this cluster.

Then using folium we can see other possibly good locations for our viennese coffee shop in the district. Avoiding the nearby competition. 